<a href="https://colab.research.google.com/github/amcheyre-nw/nlp_chatbot/blob/main/nlp_chatbot_v3__(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/nlp

/content/gdrive/MyDrive/nlp


In [3]:
! git clone https://github.com/budzianowski/multiwoz

fatal: destination path 'multiwoz' already exists and is not an empty directory.


In [4]:
! git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [7]:
!pip install rouge_score --quiet
!pip install git+https://github.com/google-research/bleurt.git --quiet
!pip install sacrebleu --quiet
!pip install git+https://github.com/huggingface/transformers --quiet
!pip install tokenizers --quiet
!pip install transformers --quiet
!pip install datasets --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [11]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
from datasets import load_metric
from pprint import pprint

import torch
import math
import time
import sys
import json
import numpy as np
import pandas as pd

In [ ]:
def make_woz_datasets(bKnowledge):
    
    if bKnowledge:
        out_names = ['woz.train_c.txt','woz.valid_c.txt','woz.test_c.txt']
    else:
        out_names = ['woz.train_b.txt','woz.valid_a.txt','woz.test_a.txt','woz.valid_b.txt','woz.test_b.txt']
    max_ins = [18,2,2,2,2]
    
    count = 0
    counts = []
    for dataset in range(len(out_names)):
        fout = open(out_names[dataset],'wt')
        for dialog in range(1,max_ins[dataset],1):
            file_name = '/content/gdrive/MyDrive/nlp/multiwoz/train/dialogues_%03d.json' % dialog
            print(file_name)
            with open(file_name) as f:
                data = json.load(f)
            for dialogue in data:
                if len(dialogue['services']) == 1:
                    if dialogue['services'][0] == 'restaurant':
                        prev_speaker = ''
                        prev_utterance = ''
                        for turn in dialogue['turns']:
                            count = count + 1
                            speaker = turn['speaker']
                            utterance = turn['utterance']
                            
                            for frame in turn['frames']:
                                if frame['service'] == 'restaurant':
                                    knowledge = ''
                                    try:
                                        knowledge = '[KNOWLEDGE] '
                                        for slot in frame['slots']:
                                            temp = '%s [EQUAL] %s [SEP] ' % (slot['slot'],slot['value'])
                                            knowledge = knowledge + temp
                                    except:
                                        nothing = 1
                                    try:
                                        if len(knowledge) == 0:
                                            knowledge = '[KNOWLEDGE] '
                                        try:
                                            intent = frame['state']['active_intent']
                                            temp = '%s [EQUAL] %s [SEP] ' % ('active_intent',intent)
                                            knowledge = knowledge + temp
                                            slot_values = frame['state']['slot_values']
                                            for slot in slot_values:
                                                vals = slot_values[slot]
                                                for val in vals:
                                                    temp = '%s [EQUAL] %s [SEP] ' % (slot,val)
                                                    knowledge = knowledge + temp
                                        except:
                                            nothing = 1
                                    except:
                                        noting = 1
                            
                            if len(prev_speaker) > 0:
                                if not bKnowledge:
                                    knowledge = ''
                                if dataset == 0:
                                    text = '[%s] %s %s [%s] %s [END]' % (prev_speaker,
                                                                         prev_utterance,
                                                                         knowledge,
                                                                         speaker,
                                                                         utterance)
                                else:
                                    text = '[%s] %s %s [%s] | %s [END]' % (prev_speaker,
                                                                         prev_utterance,
                                                                         knowledge,
                                                                         speaker,
                                                                         utterance)
                                fout.write('%s\n' % (text))
                                print(text)
                            prev_speaker = speaker
                            prev_utterance = utterance
    counts.append(count)
    count = 0
    print(counts)

In [ ]:
make_woz_datasets(True)
make_woz_datasets(False)

Streaming output truncated to the last 5000 lines.
[SYSTEM] Frankie and Bennys serves Italian food in the south part of town. Is there anything else I can help you with?  [USER] Yes, what is their phone number and where are they located at? [END]
[USER] Yes, what is their phone number and where are they located at?  [SYSTEM] Their address is Cambridge Leisure Park Clifton Way Cherry Hinton and phone number is 01223 412430 [END]
[SYSTEM] Their address is Cambridge Leisure Park Clifton Way Cherry Hinton and phone number is 01223 412430  [USER] Thank you goodbye. [END]
[USER] Thank you goodbye.  [SYSTEM] Thank you for using our system. Good bye [END]
[USER] I am looking for a restaurant that serves British food in the expensive price range.  [SYSTEM] Okay how about the Travellers Rest on the west side? [END]
[SYSTEM] Okay how about the Travellers Rest on the west side?  [USER] No. I need something in the centre. [END]
[USER] No. I need something in the centre.  [SYSTEM] Fitzbillies Restau

generate for b after making changes in sh file

In [ ]:
!bash /content/gdrive/MyDrive/nlp/fine_tune_fpt2.sh

03/14/2022 07:17:02 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/14/2022 07:17:02 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_nam

generate for c after making changes in sh file

In [ ]:
!bash /content/gdrive/MyDrive/nlp/fine_tune_fpt3.sh

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Intel MKL FATAL ERROR: Cannot load /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch_cpu.so.


In [12]:
def main(gen_mode, tuned_model):
    
    #gen_mode = 1
    gen_labels = ['logits','greedy','beam','top-p', 'top-k', 'temperature']
    #tuned_model = 1
    
    if tuned_model == 0:
        tuned = 'gpt2'
        test_name = 'woz.test_a.txt'
    elif tuned_model == 1:
        tuned = '/content/gdrive/MyDrive/nlp/b'
        test_name = 'woz.test_b.txt'
    else:
        tuned = '/content/gdrive/MyDrive/nlp/c'
        test_name = 'woz.test_c.txt'
    
    tokenizer = GPT2Tokenizer.from_pretrained(tuned)
    model = GPT2LMHeadModel.from_pretrained(tuned,pad_token_id=tokenizer.eos_token_id)
    model = model.cuda()
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])        
    print('total parameters = ',params)

    # Metrics
    metric_bleu = load_metric("bleu")
    
    print(metric_bleu)


    # # NEW METRICS ADDED
    metric_rouge = load_metric("rouge")
    metric_bleurt = load_metric("bleurt")
    # # metric_wikisplit = load_metric("wiki_split")
    
    predicts = []
    refs = []
    best = []
    bleus, bleu_best = [],[]
    # New metrics lists
    rouges, rouge_best = [],[]
    bleurts, bleurts_best = [],[]
    wikisplit = []

    max_len = 0
    total = 0
    with open(test_name,'rt') as f:
        for line in f:
            line = line.replace('\n','')
            text = line.split('|')
            prompt = text[0].strip(' ')
            in_ids = tokenizer.encode(prompt,add_special_tokens=True)
            if len(in_ids) > max_len:
                max_len = len(in_ids)
            total = total + 1
            
    max_len = max_len + 32
    print('max_len: %d total: %d' % (max_len,total))
    
    obs = 0
    with open(test_name,'rt') as f:
        for line in f:
            line = line.replace('\n','')
            text = line.split('|')
            prompt = text[0].strip(' ')
            ref = text[1].strip(' ')
            obs = obs + 1
            in_ids = tokenizer.encode(prompt,add_special_tokens=True)
            
            # Logits
            if gen_mode == 0:
                seq_len = 0
                bDone = False
                while not bDone:
                    input_ids = torch.tensor(in_ids).unsqueeze(0)
                    input_ids = input_ids.cuda()
                    outputs = model(input_ids, labels=input_ids)
                    decoded = []
                    for i in range(outputs[1].size(1)):
                        decoded.append(torch.argmax(outputs[1][0][i][:]).item())
                    decoded = torch.tensor(decoded)
                    decoded = decoded.cuda()
                    in_ids.append(decoded[decoded.size(0)-1].item())
                    input_ids = torch.tensor(in_ids).unsqueeze(0)
                    text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
                    tokens = text.split(' ')
                    if tokens[len(tokens)-1] == '[END]':
                        bDone = True
                    if len(tokens) >= max_len:
                        bDone = True
            # Greedy decoder            
            if gen_mode == 1:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                greedy = model.generate(input_ids, max_length=max_len)  
                text2 = tokenizer.decode(greedy[0], skip_special_tokens=False)
                tokens = text2.split()
            
            # Beam search decoder    
            if gen_mode == 2:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                beam = model.generate(input_ids, max_length=max_len, num_beams = 5, early_stopping = True)  
                text2 = tokenizer.decode(beam[0], skip_special_tokens=False)
                tokens = text2.split()
            
            # Top-p decoder    
            if gen_mode == 3:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                top_p = model.generate(input_ids, max_length=max_len, do_sample = True, top_p = 0.90, top_k = 0)  
                text2 = tokenizer.decode(top_p[0], skip_special_tokens=False)
                tokens = text2.split()
            
            # NEW DECODERS ADDED
            # Top-k decoder
            if gen_mode == 4:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                top_k = model.generate(input_ids, max_length=max_len, do_sample = True, top_k = 50)  
                text2 = tokenizer.decode(top_k[0], skip_special_tokens=False)
                tokens = text2.split()
            
            # Temperature decoder
            if gen_mode == 5:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                temp = model.generate(input_ids, max_length=max_len, do_sample = True, top_k = 0, temperature=0.7)  
                text2 = tokenizer.decode(temp[0], skip_special_tokens=False)
                tokens = text2.split()

                
            first = len(prompt.split(' '))
            try:
                pos_end = tokens.index('[END]')
            except:
                pos_end = len(tokens)
            try:
                pos_enduser = tokens.index('[END][USER]')
            except:
                pos_enduser = len(tokens)
            try:
                pos_endsystem = tokens.index('[END][SYSTEM]')
            except:
                pos_endsystem = len(tokens)
            last = min(pos_end,pos_enduser,pos_endsystem,len(tokens))
            predict = ' '.join(tokens[first:last])

            predictions = [predict.split()]
            references = [[ref.split()]]
            predicts.append(predictions)
            refs.append(references)

     
            try:
                
                results_bleu = metric_bleu.compute(predictions = predictions, references = references)
                results_rouge = metric_rouge.compute(predictions=predictions, references=references)
            
                bleus.append(results_bleu['bleu'])
                rouges.append(results_rouge['rougeLsum'].high.fmeasure)
                
                print('ref:  ', ref)
                print('pred: ', predict)

                # BLEU score
                if results_bleu['bleu'] > 0.01:
                    # print('ref:  ',ref)
                    # print('pred: ',predict)
                    print('Bleu[%d]: %7.3f' % (obs,results_bleu['bleu']))
                    print(' ')
                    bleu_best.append(results_bleu['bleu'])
                
                # ROUGE score
                if results_rouge['rougeLsum'].high.fmeasure > 0.01:
                    # print('ref:  ',ref)
                    # print('pred: ',predict)
                    print('Rouge[%d]: %7.3f' % (obs,results_rouge['rougeLsum'].high.fmeasure))
                    print(' ')
                    rouge_best.append(results_rouge['rougeLsum'].high.fmeasure)
                
               
            except Exception as e:
                print(e)
                continue
        if len(bleu_best)==0:
          bleus_avg = 0 
        else:
          bleus_avg = sum(bleu_best) / float(len(bleu_best))      
        print(rouge_best)     
        rouge_avg = sum(rouge_best) / float(len(rouge_best))
        print("rouge_avg",rouge_avg)
        print("bleu_aveg",bleus_avg)
        with open("prelim_results.txt", 'w') as write_file:
          write_file.write(f"\tAverage bests[BLEU=[{bleus_avg},  ROUGE={rouge_avg}]\n")


In [13]:
main(1, 1)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 96 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  are no Chinese restaurants in the cheap price range. Would you like to try something else?


In [14]:
main(2, 1)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 96 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  Garden is a chinese restaurant in the cheap price range.
Rouge[1]:   0.087
 
ref:   Yes, co

In [15]:
main(3, 1)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 96 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  sorry, there aren't any Chinese restaurants in town. Would you like to try a different cuis

In [16]:
main(4, 1)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 96 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  there are no restaurants that serve Chinese food in the cheap price range. Would you like m

In [17]:
main(5, 1)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 96 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  are three cheap Chinese restaurants: Da Vinci Pizzeria, the Lucky Star, and The Lucky Star.

In [18]:
main(1, 2)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 227 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  lucky star is a Chinese restaurant in the centre of town. Would you like their address?
Ro

In [19]:
main(2, 2)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 227 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  lucky star is in the south part of town and is in the cheap price range.
Rouge[1]:   0.138

In [20]:
main(3, 2)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 227 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  lucky star is cheap and located at Cambridge Leisure Park Clifton Way Cherry Hinton. Would

In [21]:
main(4, 2)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 227 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  lucky star serves Chinese food, is located at Cambridge Leisure Park Clifton Way Cherry Hi

In [22]:
main(5, 2)

total parameters =  124439808
Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_leng

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 227 total: 511
ref:   I've heard good things about the lucky star. Need a reservation? [END]
pred:  is lucky star, it is cheap and in the west part of town.
Rouge[1]:   0.154
 
ref:   Yes, c